In [1]:
import bs4
from bs4 import BeautifulSoup as soup
import requests
import pandas as pd
import time
import random
from IPython.display import clear_output, display

REQUESTS_PER_MINUTE = 20
wpa_df = pd.DataFrame

In [3]:
teams = ["ARN", "ATL", "BAL", "BOS","CHA","CHN","CIN","CLE","COL","DET", "HOU", "KCA", "LAA", "LAN", "MIA", "MIL", "MIN", "NYA", "NYN", "OAK", "PHI", "PIT", "SDN", "SFN", "SEA", "STL", "TBA", "TEX", "TOR","WAS"]

In [4]:
def scrape(url, gameID, ds):
    time.sleep(60/REQUESTS_PER_MINUTE)
    req = requests.get(url)
    resp = req.status_code
    d = soup(req.text, 'html.parser')
    if resp == 200:
        block = [i for i in d.find_all(string=lambda text: isinstance(text, bs4.Comment)) if 'id="div_play_by_play"' in i][0]
        b = soup(str(block), 'html.parser')
        table = b.find_all('table')
        path = f"./wpa_data/{gameID}.csv"
        df = pd.read_html(str(table))[0]
        df.to_csv(path)
        data = ingestDF(df, gameID, path, ds)
        return data
    else:
        return None

In [11]:
days = list(range(1,32))
months = list(range(3,10))
total_queries = len(days) * len(months) * len(teams) * 2
count = 0

full_data = []
for teamID in teams:
    for month in months:
        for day in days:
            for i in range(2):
                gameID = f"{teamID}2023{str(month).zfill(2)}{str(day).zfill(2)}{i}"
                date_string = f"2023{str(month).zfill(2)}{str(day).zfill(2)}{i}"
                url = f"https://www.baseball-reference.com/boxes/{teamID}/{gameID}.shtml"
                data = scrape(url, gameID, date_string)
                clear_output(wait=True)
                display(f"{count}/{total_queries} ({(count/total_queries)*100:.2f}%) queries complete, just requested {gameID}")
                count += 1
                if data is not None:
                    full_data.append(data)
pd.DataFrame(full_data).to_csv("full_data.csv")

'120/13020 (0.92%) queries complete, just requested ARN202304300'

KeyboardInterrupt: 

In [24]:
def getTeamName(df, away):
    if away:
        return(df[df.Inn=="t1"]["@Bat"].iloc[0])
    else:
        return(df[df.Inn=="b1"]["@Bat"].iloc[0])

In [38]:
for f in range(10):
  # use display(f) if you encounter performance issues
    time.sleep(1)

9